In [5]:
import logging
logging.getLogger("matplotlib").setLevel(logging.INFO)
logging.getLogger("numba").setLevel(logging.INFO)
logging.getLogger("jieba").setLevel(logging.INFO)
import matplotlib.pyplot as plt
import IPython.display as ipd
from mel_processing import spectrogram_torch
import os

import torch
from torch.utils.data import DataLoader
from utils import load_wav_to_torch

import utils
from models import SynthesizerTrn
from text.symbols import symbols
from text.cleaner import text_to_sequence

# logging.getLogger("matplotlib").setLevel(logging.INFO)
# logging.getLogger("matplotlib").setLevel(logging.INFO)
def get_text(text):
    text_norm = text_to_sequence(text)
    text_norm = torch.LongTensor(text_norm)
    return text_norm
hps = utils.get_hparams_from_file("./configs/new.json")
net_g = SynthesizerTrn(
        len(symbols),
        hps.data.filter_length // 2 + 1,
        hps.data.hop_length,
        hps.data.sampling_rate,
        hps.train.segment_size // hps.data.hop_length,
        n_speakers=hps.data.n_speakers,
        **hps.model)
_ = net_g.eval()

_ = utils.load_checkpoint("/Volumes/Extend/下载/gstG_7000 (1).pth", net_g, None)

load 
INFO:root:Loaded checkpoint '/Volumes/Extend/下载/gstG_7000 (1).pth' (iteration 5)


In [6]:
# 使用scipy.io.wavfile.read读取的音频文件
def get_spepc(filename):
    audio, sampling_rate = load_wav_to_torch(filename)
    if sampling_rate != hps.data.sampling_rate:
        raise ValueError("{} SR doesn't match target {} SR".format(
            sampling_rate, hps.data.sampling_rate))
    audio_norm = audio / hps.data.max_wav_value
    audio_norm = audio_norm.unsqueeze(0)
    spec = spectrogram_torch(audio_norm, hps.data.filter_length,
        hps.data.sampling_rate, hps.data.hop_length, hps.data.win_length,
        center=False)
    return spec

In [7]:
# stn_tst = get_text("大家好啊,我是说的道理")
# with torch.no_grad():
#     x_tst = stn_tst.unsqueeze(0)
#     x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
#     sid = torch.LongTensor([1])
#     audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667,spec=None)[0][0,0].data.cpu().float().numpy()
# ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

In [18]:
from utils import load_wav_to_torch
from GST import Hyperparameters as hparams
GST = torch.tanh(net_g.gst.stl.embed)

for idx in range(2):
    query = torch.zeros(1, 1, hparams.E//2)
    keys = GST[idx].unsqueeze(0).expand(1,-1,-1)
    style_emb = net_g.gst.stl.attention(query, keys)
    stn_tst = get_text("[JA]重症 特定 うまく もう 半分 ぐらい まで 特定 さ れ て たもう もう 引っ越し た から あれ なん だけ ぶっちゃ きる ん だ けど [JA]")
    with torch.no_grad():
        x_tst = stn_tst.unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
        audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667,g=style_emb.transpose(1,2), spec=None)[0][0,0].data.cpu().float().numpy()
    ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))




In [11]:

spec = get_spepc("/Volumes/Extend/下载/vispeechdata/serena/wavs/000003.wav")
stn_tst = get_text("[JA]重症 特定 うまく もう 半分 ぐらい まで 特定 さ れ て たもう もう 引っ越し た から あれ なん だけ ぶっちゃ きる ん だ けど [JA]")
with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, spec=spec)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))





### Voice Conversion

In [ ]:
dataset = TextAudioSpeakerLoader(hps.data.validation_files, hps.data)
collate_fn = TextAudioSpeakerCollate()
loader = DataLoader(dataset, num_workers=8, shuffle=False,
    batch_size=1, pin_memory=True,
    drop_last=True, collate_fn=collate_fn)
data_list = list(loader)

In [ ]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x.cuda() for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([1]).cuda()
    sid_tgt2 = torch.LongTensor([2]).cuda()
    sid_tgt3 = torch.LongTensor([4]).cuda()
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    audio2 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.cpu().float().numpy()
    audio3 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps.data.sampling_rate, normalize=False))